In [ ]:
import os
import gym
import numpy as np
import cv2
import time
from gym.spaces import Box
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack

# ==========================================
# 1. CONFIGURAZIONE
# ==========================================
MODEL_PATH = "./train/ttlivelli/mario_model_VITTORIA.zip" 

# ==========================================
# 2. DEFINIZIONE AMBIENTE (CON FIX PER BLOCCO)
# ==========================================
class MarioModernized(gym.Env):
    def __init__(self):
        super().__init__()
        inner_env = gym_super_mario_bros.make('SuperMarioBros-1-2-v0')
        self.mario = JoypadSpace(inner_env, SIMPLE_MOVEMENT)
        self.observation_space = Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8)
        self.action_space = self.mario.action_space
        
        # Variabili per gestire il blocco
        self.max_x = 0
        self.stagnant_steps = 0
        self.skip_frames = 4 

    def _process_frame(self, frame):
        if frame is not None:
            gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
            resized = cv2.resize(gray, (84, 84), interpolation=cv2.INTER_AREA)
            
            # Visualizzazione memoria
            if not hasattr(self, 'view_stack'):
                self.view_stack = [np.zeros((84, 84), dtype=np.uint8)] * 4
            self.view_stack.pop(0)
            self.view_stack.append(resized)
            film_strip = np.hstack(self.view_stack)
            big_strip = cv2.resize(film_strip, (84*4*2, 84*2), interpolation=cv2.INTER_NEAREST)
            cv2.imshow("Visione IA (4 Frame)", big_strip)
            cv2.waitKey(1)
            
            return resized[:, :, np.newaxis]
        return np.zeros((84, 84, 1), dtype=np.uint8)

    def reset(self, seed=None, options=None):
        self.max_x = 0
        self.stagnant_steps = 0
        obs = self.mario.reset()
        self.mario.render()
        return self._process_frame(obs)

    def step(self, action):
        # Eseguiamo l'azione per 4 frame
        for _ in range(self.skip_frames):
            obs, reward, done, info = self.mario.step(action)
            if done: break

        self.mario.render()
        
        # --- FIX: CONTROLLO SE MARIO È BLOCCATO ---
        x_pos = info['x_pos']
        if x_pos > self.max_x:
            self.max_x = x_pos
            self.stagnant_steps = 0
        else:
            self.stagnant_steps += 1
        
        # Se Mario non avanza per 100 decisioni (circa 6-7 secondi di gioco reale)
        if self.stagnant_steps > 100:
            print("MARIO BLOCCATO: Reset forzato...")
            done = True # Forza la fine dell'episodio
        
        if info['flag_get']:
            print("!!! BANDIERA RAGGIUNTA !!!")

        obs = self._process_frame(obs)
        return obs, reward, done, info

# ==========================================
# 3. ESECUZIONE
# ==========================================
if __name__ == "__main__":
    if not os.path.exists(MODEL_PATH):
        print(f"ERRORE: Modello non trovato in {MODEL_PATH}")
        exit()

    # Creazione ambiente identico al training
    env = MarioModernized()
    env = DummyVecEnv([lambda: env])
    env = VecFrameStack(env, n_stack=4, channels_order='last')

    print("Caricamento modello...")
    model = PPO.load(MODEL_PATH, env=env)
    
    obs = env.reset()
    while True:
        # Predict
        action, _ = model.predict(obs, deterministic=False)
        obs, reward, done, info = env.step(action)
        
        # NOTA: DummyVecEnv resetta automaticamente quando done=True,
        # ma stampiamo per chiarezza
        if done:
            print("Fine partita. Ricomincio...")
            # Non serve chiamare env.reset() manualmente con DummyVecEnv, 
            # ma lo facciamo per essere sicuri che i contatori della nostra classe si azzerino
            obs = env.reset()

Caricamento modello...
Wrapping the env in a VecTransposeImage.


C:\Users\matte\Desktop\marioia\mario_stabile\Lib\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning: overflow encountered in scalar subtract
  return (self.ram[0x86] - self.ram[0x071c]) % 256


MARIO BLOCCATO: Reset forzato...
Fine partita. Ricomincio...
Fine partita. Ricomincio...
MARIO BLOCCATO: Reset forzato...
Fine partita. Ricomincio...
Fine partita. Ricomincio...
Fine partita. Ricomincio...
Fine partita. Ricomincio...
Fine partita. Ricomincio...
Fine partita. Ricomincio...
Fine partita. Ricomincio...
Fine partita. Ricomincio...
Fine partita. Ricomincio...
Fine partita. Ricomincio...
Fine partita. Ricomincio...
Fine partita. Ricomincio...
Fine partita. Ricomincio...
Fine partita. Ricomincio...
Fine partita. Ricomincio...
Fine partita. Ricomincio...
Fine partita. Ricomincio...
Fine partita. Ricomincio...
Fine partita. Ricomincio...
Fine partita. Ricomincio...
MARIO BLOCCATO: Reset forzato...
Fine partita. Ricomincio...
MARIO BLOCCATO: Reset forzato...
Fine partita. Ricomincio...
Fine partita. Ricomincio...
MARIO BLOCCATO: Reset forzato...
Fine partita. Ricomincio...
MARIO BLOCCATO: Reset forzato...
Fine partita. Ricomincio...
